
    This Source Code Form is subject to the terms of the Mozilla Public
    License, v. 2.0. If a copy of the MPL was not distributed with this
    file, You can obtain one at http://mozilla.org/MPL/2.0/.


In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.transforms
import numpy as np
import pandas as pd
import seaborn as sns

### Global parameter to build seaborn boxplot

In [ ]:
json_data_from_lcs_1 = 'data/bacs-2-refactor.json'

json_data_from_lcs_2 = 'data/bacs-2-pees.json'

figure_size = (20, 10)

axe_label_fontsize = '24'
tick_label_fontsize = '20'
legend_fontsize = '20'

### Reading json files and preparing data

In [ ]:
with open(json_data_from_lcs_1) as json_file_1:
    raw_data_from_lcs_1 = json.load(json_file_1)
    
with open(json_data_from_lcs_2) as json_file_2:
    raw_data_from_lcs_2 = json.load(json_file_2)
    
cleaned_data = {'LCS':[], 'Maze':[], 'Distance to exit':[], 'Knowledge':[], 'Knowledge_zip':[], 'Population':[], 'Reliable':[], 'Population_zip':[], 'Reliable_zip':[], 'Zipping_rate':[], 'Zipping_loss':[]}
raw_data = [raw_data_from_lcs_1,raw_data_from_lcs_2]
lcs_name = [json_data_from_lcs_1[5:-5], json_data_from_lcs_2[5:-5]]

nb_of_environments = 0

for i in range(len(raw_data)):
    for item in raw_data[i]:
        if 'time' not in item.keys():
            nb_of_environments += 1
            for idx, value in enumerate(item['avg_exploit_rl_list']):
                cleaned_data['LCS'].append(lcs_name[i])
                cleaned_data['Maze'].append(item['maze'])
                cleaned_data['Distance to exit'].append(value)
                cleaned_data['Knowledge'].append(item['knowledge_list'][idx])
                cleaned_data['Knowledge_zip'].append(item['knowledge_list_compressed'][idx])
                cleaned_data['Population'].append(item['population_list'][idx])
                cleaned_data['Reliable'].append(item['reliable_list'][idx])
                cleaned_data['Population_zip'].append(item['population_list_compressed'][idx])
                cleaned_data['Reliable_zip'].append(item['reliable_list_compressed'][idx])
                cleaned_data['Zipping_rate'].append(1. - float(item['population_list_compressed'][idx])/item['population_list'][idx])
                cleaned_data['Zipping_loss'].append(item['knowledge_list'][idx] - item['knowledge_list_compressed'][idx])

nb_of_environments /= len(lcs_name)
                
pandas_data = pd.DataFrame(cleaned_data)

### Plotting boxplots with seaborn

In [ ]:
rc={
    'axes.labelsize':axe_label_fontsize,
    'xtick.labelsize':tick_label_fontsize,
    'ytick.labelsize':tick_label_fontsize
}
sns.set(context='notebook', style='ticks', rc=rc)

In [ ]:
def plot_figure(pandas_data, Y, legend_loc):
    
    # Build the main figure
    fig, ax = plt.subplots(figsize=figure_size)
    sns.stripplot(
        x = 'Maze',
        y = Y,
        hue = 'LCS',
        data = pandas_data,
        edgecolor='gray',
        jitter=True,
        linewidth = 1,
        dodge=True,
        palette='pastel',
        ax = ax
    )
    sns.boxplot(
        x = 'Maze',
        y = Y,
        hue = 'LCS',
        data = pandas_data,
        fliersize=0,
        palette='pastel',
        ax = ax
    )

    # Set up x tick labels and x label correctly
    plt.xticks(rotation = 45, horizontalalignment = 'right')
    plt.xlabel('')

    # Hide the horizontal gridlines
    ax.yaxis.grid(False)

    # Show the vertical gridlines
    ax.xaxis.grid(True)

    # Shift major ticks to create column for each maze
    plt.xticks(np.arange(nb_of_environments)+0.5)

    # Set up minor ticks and align label of major ticks on minor ticks
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.5))
    dx = (((figure_size[0] * 100 / 4.))/nb_of_environments - 10)/72.; dy = 0/72. # Width in inches converted into pixels depending on default DPI
    offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    for label in ax.xaxis.get_majorticklabels():
        label.set_transform(label.get_transform() - offset)

    # Set up legend
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles[0:2], labels[0:2], frameon=False, loc=legend_loc, fontsize=legend_fontsize)

In [ ]:
def plot_distance_to_exit(pandas_data):
    
    plot_figure(pandas_data, Y='Distance to exit', legend_loc='upper right')

In [ ]:
def plot_knowledge(pandas_data, zipped:bool=True):
    
    if zipped:
        Y='Knowledge_zip'
    else:
        Y='Knowledge'
        
    plot_figure(pandas_data, Y=Y, legend_loc='lower right')

In [ ]:
def plot_population(pandas_data, only_reliable:bool=False, zipped:bool=True):
    
    if not only_reliable:
        if zipped:
            Y = 'Population_zip'
        else:
            Y = 'Population'
    else:
        if zipped:
            Y = 'Reliable_zip'
        else:
            Y = 'Reliable'
            
    plot_figure(pandas_data, Y=Y, legend_loc='upper right')

In [ ]:
def plot_zipping_rate(pandas_data):
    
    plot_figure(pandas_data, Y='Zipping_rate', legend_loc='lower right')

In [ ]:
def plot_zipping_loss(pandas_data):
    
    plot_figure(pandas_data, Y='Zipping_loss', legend_loc='upper right')

In [ ]:
plot_distance_to_exit(pandas_data)

In [ ]:
plot_knowledge(pandas_data)

In [ ]:
plot_population(pandas_data, only_reliable=False, zipped=False)
plot_population(pandas_data, only_reliable=False, zipped=True)

In [ ]:
plot_zipping_rate(pandas_data)

In [ ]:
plot_zipping_loss(pandas_data)